In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'/content/drive/MyDrive/CNN.project3.ipynb'

In [ ]:
# # Organize data into train, valid, test dirs
os.chdir('/content/drive/MyDrive/CNN.project2/CNN.project2/CNN.project3.ipynb')
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')

    for c in random.sample(glob.glob('cat*'), 500):
        shutil.move(c, 'train/cat')      
    for c in random.sample(glob.glob('dog*'), 500):
        shutil.move(c, 'train/dog')
    for c in random.sample(glob.glob('cat*'), 100):
        shutil.move(c, 'valid/cat')        
    for c in random.sample(glob.glob('dog*'), 100):
        shutil.move(c, 'valid/dog')
    for c in random.sample(glob.glob('cat*'), 50):
        shutil.move(c, 'test/cat')      
    for c in random.sample(glob.glob('dog*'), 50):
        shutil.move(c, 'test/dog')
os.chdir('../../')


In [ ]:
train_path='/content/drive/MyDrive/CNN.project2/CNN.project2/CNN.project3.ipynb/data dogs-vs-cats/train'
valid_path='/content/drive/MyDrive/CNN.project2/CNN.project2/CNN.project3.ipynb/data dogs-vs-cats/valid'
test_path='/content/drive/MyDrive/CNN.project2/CNN.project2/CNN.project3.ipynb/data dogs-vs-cats/test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
    
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

In [ ]:
train_batches.n == 1000
valid_batches.n ==200
test_batches.n ==100
train_batches.num_classes==valid_batches.num_classes==test_batches.num_classes==2

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)

#Build A Simple CNN

In [ ]:
model = Sequential([
    Conv2D(filters=64,kernel_size=(3, 3),activation='relu',padding='same',input_shape=(224, 224, 3)),
    MaxPool2D(pool_size=(2, 2),strides=2),
    Conv2D(filters=128, kernel_size=(3, 3),activation='relu',padding='same'),
    Flatten(),
    Dense(units=2,activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

#predict
-We’ll now run next(test_batches) to extract a batch of images and their corresponding labels from the test set.

In [ ]:
test_imgs, test_lables= next(test_batches)
plotImages(test_imgs)
print(test_lables)

In [ ]:
test_batches.classes

In [ ]:
predictions= model.predict(x=test_batches, verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
cm =confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions,axis=1))

In [ ]:
cm

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')                       

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

#Data Augmentation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
gen = ImageDataGenerator(rotation_range=10,width_shift_range=0.1,
      height_shift_range=0.1, shear_range=0.15,zoom_range=0.1,
      channel_shift_range=10., horizontal_flip=True)

In [ ]:
chosen_image = random.choice(os.listdir('/content/drive/MyDrive/CNN.project2/CNN.project2/CNN.project3.ipynb/data dogs-vs-cats/train/dog/'))

In [ ]:
image_path = '/content/drive/MyDrive/CNN.project2/CNN.project2/CNN.project3.ipynb/data dogs-vs-cats/train/dog/' + chosen_image

In [ ]:
os.path.isfile(image_path)

In [ ]:
image = np.expand_dims(plt.imread(image_path),0)
plt.imshow(image[0]);

In [ ]:
# Generator batches augmentedd from this image
aug_iter = gen.flow(image)

In [ ]:
#Get 10 augmented images
aug_image = [next(aug_iter)[0].astype(np.uint8)for i in range(10)]

In [ ]:
#Augmented images
plotImages(aug_image)

#Build Fine-tuned VGG16 model

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

We loop in all the output layers of the last layer and then we add each layer in a new sequence.

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
type(vgg16_model)

In [ ]:
for layer in model.layers:
    layer.trainable = False

Now remember that we have removed the output layer, which has 10 output curves. And it had 1000 output classes. Now we want to add two output layers that have only two output classes for cats and dogs.

In [ ]:
model.add(Dense(units=2, activation='softmax'))

So we see that everything is the same except Dense

In [ ]:
model.summary()

# train the fine-tuned vgg16 model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=5, verbose=2)

#predict the fine-tuned vgg16 model

In [ ]:
predictions = model.predict(x=test_batches,  verbose=0)

In [ ]:
test_batches.classes

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(predictions, axis=-1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels =['cat', 'dog']
plot_confusion_matrix(cm= cm,classes=cm_plot_labels, title='Confusion Matrix')